### SAT Dashboard 
* [#177](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/177)


In [3]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-09-20 23:56:32


In [6]:
def _render_intro_html(intro_html: str, mode: str = "off") -> str:
    """
    mode:
      - "off"         -> render nothing (default)
      - "inline"      -> tiny inline link bar (no big panel, minimal spacing)
      - "collapsible" -> <details> with a 'Links' summary (closed by default)
      - "panel"       -> compact panel (if you still want a box)
    """
    if not intro_html or mode == "off":
        return ""

    if mode == "inline":
        return f'''
        <div class="sat-links" style="margin:4px 0 6px; font-size:.92rem;
             display:flex; flex-wrap:wrap; gap:.5rem; align-items:center;">
          {intro_html}
        </div>
        <style>
          .sat-links a {{ color:#2563eb; text-decoration:none }}
          .sat-links a:hover {{ text-decoration:underline }}
          .sat-links p {{ margin:0 }}
        </style>
        '''

    if mode == "collapsible":
        return f'''
        <details class="sat-links" style="margin:2px 0">
          <summary style="cursor:pointer; font-weight:600">Links</summary>
          <div style="margin-top:6px; display:flex; flex-wrap:wrap; gap:.5rem;">
            {intro_html}
          </div>
        </details>
        <style>
          .sat-links a {{ color:#2563eb; text-decoration:none }}
          .sat-links a:hover {{ text-decoration:underline }}
          .sat-links p {{ margin:.2rem 0 }}
        </style>
        '''

    # fallback: compact panel (kept for completeness)
    return f'''
    <section class="sat-panel" style="padding:6px 0">
      <div class="sat-intro">{intro_html}</div>
    </section>
    <style>
      .sat-intro p{{margin:.2rem 0}}
      .sat-intro a{{color:#2563eb;text-decoration:none}}
      .sat-intro a:hover{{text-decoration:underline}}
    </style>
    '''


In [8]:
def make_sat_dashboard(
    OUTPUT_DIR: str,
    PROJECT_NAME: str,
    stamp: str,
    *,
    map_gallery: list[dict] | None = None,
    intro_html: str = "",                 
    charts_blocks: list[dict] | None = None,
    summary_html: str = "",
) -> str:
    gallery_html = _render_map_gallery_html(map_gallery or [])
    intro_block  = _render_intro_html(intro_html)              # NEW
    charts_html  = _render_charts_html(charts_blocks or [])    # NEW

    html_out = f"""
<!doctype html>
<html lang="en">
<head>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <meta charset="utf-8">
  <title>SAT Dashboard — where do we walk on Stockholm Archipelago Trail</title>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
  <style>
    body{{margin:0;padding:20px;background:#f8fafc;font:14px/1.4 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;color:#111827}}
    .sat-wrap{{max-width:1200px;margin:0 auto}}
    .sat-hero{{background:#111827;border-radius:14px;color:white;padding:16px 18px;margin-bottom:16px;display:flex;flex-wrap:wrap;align-items:center;gap:10px}}
    .sat-hero h1{{margin:0;font-size:20px}}
    .sat-hero a{{color:#a5b4fc;text-decoration:none}}
    .sat-row{{display:grid;gap:14px;grid-template-columns:1fr}}
    @media(min-width:960px){{ .sat-row{{grid-template-columns:2fr 1fr}} }}
    .sat-panel{{background:#fff;border-radius:12px;box-shadow:0 2px 10px rgba(0,0,0,.06);padding:12px}}
    .muted{{color:#6b7280}}
    .sat-card{{ position:relative; }}
    .issues{{ position:absolute; top:10px; right:10px; display:flex; gap:6px; align-items:center; z-index:2; }}
    .issue-icon{{ font-size:18px; line-height:1; color:#374151; text-decoration:none; }}
    .issue-icon:hover{{ color:#111827; }}
    .issue-badge{{
      font:600 12px/1 system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Helvetica,Arial,sans-serif;
      padding:4px 6px; border-radius:999px; background:#f3f4f6; color:#111827; text-decoration:none; border:1px solid #e5e7eb;
    }}
    .issue-badge:hover{{ background:#e5e7eb; }}
  </style>
</head>
<body>
  <div class="sat-wrap">
    <div class="sat-hero">
      <h1>SAT Dashboard — where do we walk on Stockholm Archipelago Trail</h1>
      <div class="muted" style="margin-left:auto">Updated: {html.escape(stamp)}</div>
    </div>

    {intro_block}     
    {gallery_html}

    <div class="sat-row">
      <div class="sat-panel">
        {charts_html}   <!-- NEW: chart panels with descriptions -->
      </div>
      <div class="sat-panel">
        {summary_html}
      </div>
    </div>
  </div>
</body>
</html>
"""

    # save + latest, same as before
    out_dir = Path(OUTPUT_DIR); out_dir.mkdir(parents=True, exist_ok=True)
    html_path = out_dir / f"{PROJECT_NAME}_dashboard_{stamp}.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_out)
    latest_path = _to_latest_name(html_path)
    shutil.copyfile(html_path, latest_path)
    print(f"Saved: {html_path}\nUpdated: {latest_path}")
    return str(html_path)

In [9]:
summary_html = _render_intro_html("""
<ul style="list-style-type: none; padding-left: 0; font-family: Arial, sans-serif; font-size:14px; line-height:1.6;">
  <li style="margin-bottom:6px;">🔗 Projektyta <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue" target="_blank">GITHUB</a></li>
  <li style="margin-bottom:6px;">🖼️ <a href="https://commons.wikimedia.org/wiki/Category:Stockholm%20Archipelago%20Trail" target="_blank">Bilder på leden</a> – Wikicommons - <a href="https://wikimap.toolforge.org/?cat=Stockholm_Archipelago_Trail&subcats=true&subcatdepth=4&cluster=false">Karta (tar tid)</a></li>
  <li style="margin-bottom:6px;">👣 Icke officiell <a href="https://www.facebook.com/groups/2875020699552247" target="_blank">FB grupp om leden av vandrare för vandrare</a></li>
  <li style="margin-bottom:6px;">📚 <a href="https://www.wikidata.org/wiki/Wikidata:WikiProject_Stockholm_Archipelago_Trail" target="_blank">Wikipedia projekt</a> / Umap: <a target=_blank href="https://umap.openstreetmap.fr/en/map/boende-nordsydlinjen-och-stockholm-archipelago-tra_1257362?scaleControl=true&miniMap=true&scrollWheelZoom=true&zoomControl=true&editMode=disabled&moreControl=true&searchControl=true&tilelayersControl=true&embedControl=null&datalayersControl=true&onLoadPanel=caption&captionBar=true&captionMenus=true&datalayers=35da97a6-893b-46de-b7b3-4ed341b042c3%2Cb3a4a332-36f7-4949-80dd-2468d4a712ea%2C752d2480-e5a5-4fa2-9b78-15c75e545f64%2C7b203db0-53d4-4a1a-9f7d-a298344c6da5&locateControl=true&measureControl=true">NordSyd</a>, <a target=_blank href="https://umap.openstreetmap.fr/en/map/stockholm-archipelago-trail-naturreservat-talta_1280785?scaleControl=false&miniMap=false&scrollWheelZoom=false&zoomControl=true&editMode=disabled&moreControl=true&searchControl=null&tilelayersControl=null&embedControl=null&datalayersControl=true&onLoadPanel=none&captionBar=false&captionMenus=true">Tälta</a></li>
  <li style="margin-bottom:6px;">🔗 Video om denna yta <a href="https://youtu.be/vy_746Wn4pc" target="_blank">youtube</a> / <a target="_blank" href="https://www.youtube.com/playlist?list=PLNWUKRLAYDeSSHsFOAOy8L_cAtbsOQ41R">SAT spelista</a></li>
  <li style="margin-bottom:6px;">🔗 Samtal med: <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/146" target="_blank">Samtal med: Tillväxtverket</a>, <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/174" target="_blank">Naturvårdsverket</a> med dom problem vi ser att låta Visit Sweden driva projekt...
</ul>
""")

gallery = [
    {
        "title": "Koppling mellan led och bild – kvalitetskontroll - Image Quality Control",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_image_qc_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Image_Quality_Control_400x225.jpg",  # optional if you have it
        "desc": "Hela leden är dokumenterad i OpenStreetMap (OSM), där varje etapp och delsträcka kopplas till bilder på Wikimedia Commons. Denna karta gör det möjligt att kvalitetssäkra både kartinformationen och bildmaterialet längs leden. Två kartlager finns med toaletter och dricksvatten.",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"
        ]
    },
    {
        "title": "Funktionstillgänglighet – rullstol - Wheelchair",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_WHEELCHAIR_073_wheelchair_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Wheelchair_thumbnail.jpg",
        "desc": "Visar objekt längs leden som är markerade med taggen 'wheelchair' i OpenStreetMap.",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/73"
        ]
    },
    {
        "title": "Att göra - Todo",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_ALL_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Audit_layers_thumbnail.jpg",
        "desc": "Visualiserar saknade OSM-taggar på en karta över leden",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"
        ]
     },
    {
        "title": "Audit lager",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_split_todo_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Audit_layers_thumbnail.jpg",
        "desc": "Karta som visar var saknas OSM taggarma surface / foot / sac / trail_visibility / step_count",
         "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142"
        ]
     },
    {
        "title": "Trappsteg på leden - Steps",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_steps_only_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Steps_Only_thumbnail.jpg",
        "desc": "SAT leden har begärt ersättning för att bygga 200 meter trappor dom är svåra att hitta",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/148"
        ]
    },
    {
        "title": "Närhet till vatten - Proximity",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_water_proximity_latest.html",
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Water_Proximity_800x450.jpg",
        "desc": "En observation längs leden är att vissa sträckor går långt från vattnet, på både gott och ont. Denna karta visar hur nära eller långt från vattnet olika delar av leden ligger, samt vilken sträcka som är längst bort.",
    },
    {
        "title": "Wikipedia/Wikidata",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_Wikipedia.html",                                                                                                 
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_Wikipedia_400x225.jpg",
        "desc": "Mer än 600 Wikipedia/Wikidata objekt har kopplats till SAT leden",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/22"
        ]
    },
    {
        "title": "Toaletter nära leden",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_24_18_59.html",                                                                                                 
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_132_toilets.jpg",
        "desc": "Söker i Open Street Map efter toaletter längs leden och ser vilket metadata som saknas ",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132"
        ]
    },
    {
        "title": "Dricksvatten nära leden",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_latest.html",                                                                                                 
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_139_drinkingwater.jpg",
        "desc": "Söker i Open Street Map efter dricksvatten längs leden idag saknas ett ekosystem där man digitalt kan se vad som är påslaget, testat...",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139"
        ]
    },
    {
        "title": "SAT infoskyltar",
        "url": "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_176_signs_latest.html",                                                                                                 
        "img": "https://raw.githubusercontent.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/thumbs/SAT_Map_176_signs.jpg",
        "desc": "135 turistinformationsskyltar skall vara beställda av Tillväxtverket endast 80 har hittats, ingen öppen data om var dom finns har levererats ännu...",
        "issues": [
            "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/176"
        ]
    }
    ]


NameError: name 'etapp_gdfs' is not defined

In [ ]:
OUTPUT_DIR=OUTPUT_DIR,

dash_html = make_sat_dashboard(
        OUTPUT_DIR=OUTPUT_DIR,
        PROJECT_NAME=PROJECT_NAME,
        stamp=stamp,
        intro_html=summary_html,
        summary_html=summary_html,
        map_gallery=gallery
    )
print("✅ Dashboard with gallery =>", dash_html)

In [10]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-09-20 23:59:04
Total time elapsed: 02 minutes 32.21 seconds
